In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
from wordcloud import WordCloud
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id = "1"></a>
## Load And Check Data

In [ ]:
data = pd.read_csv("/kaggle/input/ufo-sightings/scrubbed.csv")

In [ ]:
data.info()

float64(1): longitude <br>
object(10): datetime, city, state, country, shape, duration(seconds), duration(hours/min), date posted, latitude

In [ ]:
data.head(5).T

In [ ]:
print("Unique states:")
print(data.state.unique())
print("Proportion of NaNs: %.2f" % (sum(data.country.isnull())/data.shape[0]))
print("Unique shapes:")
print(data['shape'].unique())
print("Proportion of NaNs: %.2f" % (sum(data['shape'].isnull())/data.shape[0]))

In [ ]:
def custom_to_datetime(date):
    if date[11:13] == '24':
        x = date[:11] + '23:59'
    elif date[10:12] == '24':
        x = date[:10] + '23:59'
    elif date[9:11] == '24':
        x = date[:9] + '23:59'
    else:
        return pd.to_datetime(date)
    return pd.to_datetime(x)

data.index = np.arange(0, len(data))
data['datetime'] = data['datetime'].apply(custom_to_datetime)  
data['date posted'] = pd.to_datetime(data['date posted'])

data['country'] = data['country'].fillna('unknown')
data['state'] = data['state'].fillna('unknown')
data['shape'] = data['shape'].fillna('unknown')
data['comments'] = data['comments'].fillna('unknown')

data['shape'] = data['shape'].astype('category')
data['state'] = data['state'].astype('category')
data['country'] = data['country'].astype('category')

data['duration (seconds)'] = pd.to_numeric(data['duration (seconds)'], errors='coerce')
data['latitude'] = pd.to_numeric(data['latitude'], errors='coerce')
data['longitude '] = pd.to_numeric(data['longitude '], errors='coerce')    

data.loc[: ,'latitude'].replace({0.0: np.nan}, inplace=True)
data.loc[: ,'longitude '].replace({0.0: np.nan}, inplace=True)
data.loc[: ,'duration (seconds)'].replace({0.0: np.nan}, inplace=True)
#data['month'] = data['datetime'].dt.month
#data['weekday'] = data['datetime'].dt.weekday
data['month'] = pd.DatetimeIndex(data['datetime']).month #creates column 'month' stripping the year from datetime_zero
data['year'] = pd.DatetimeIndex(data['datetime']).year #creates column 'year' stripping the year from datetime_zero
data['day'] = pd.DatetimeIndex(data['datetime']).day #creates column 'day' stripping the year from datetime_zero
data['hour'] = pd.DatetimeIndex(data['datetime']).hour #creates column 'hour' stripping the year from datetime_zero


In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

In [ ]:
data.head(100).T

In [ ]:
data = data.dropna(thresh=8)

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
##Yıllara göre görülme sıklığı
years_data = data['year'].value_counts()
years_index = years_data.index  
years_values = years_data.values
plt.figure(figsize=(15,8))
plt.xticks(rotation = 60)
plt.title('UFO Sightings since 1943', fontsize=18)
plt.xlabel("Year", fontsize=14)
plt.ylabel("Number of reports", fontsize=14)
years_plot = sns.barplot(x=years_index[:80],y=years_values[:80], palette = "Blues_d")

## 

* 2000'li yılların başlaması ile birlikte görülen ufo sayılarında ciddi bir artış gözlemlenmiş. Fakat 2014 yılında keskin bir düşüş gözlemleniyor. 2014 yılının ortalararında çalışma durdurulmuş. 

In [ ]:
## Aylara göre ufo report sıklıkları
order = ["January", "February", "March", "April", "May", "June", "July","August", "September", 
         "October", "November", "December"]
data['month'] = data['datetime'].dt.month_name() #turns month numbers into month names
month_data = data["month"].value_counts()
month_index = month_data.index  
month_values = month_data.values
plt.figure(figsize=(15,8))
plt.title('UFO Sightings by Month', fontsize=18)
plt.xlabel("Month",fontsize=14)
plt.ylabel("Number of reports", fontsize=14)
month_plot = sns.barplot(x=month_index[:60],y=month_values[:60], palette = "Blues", order=order)



*  En fazla haziran,temmuz ve ağustos aylarında ufo gözlem raporlaması var. Yaz aylarında daha çok raporlama olduğu ortaya çıkıyor. Bunun sebebi sıcak havalarda daha fazla insanın dışarı çıkması olabilir. 

In [ ]:
# Ayların günlerine bakacak olursak en çok ayın 15inde ve 1inde raporlanmış.
day_data = data['day'].value_counts() 
day_index = day_data.index 
day_values =day_data.values
plt.figure(figsize=(15,8))
plt.title('UFO Sightings by Day', fontsize=18)
plt.xlabel("Day of the Month", fontsize=14)
plt.ylabel("Number of reports", fontsize=14)
day_plot = sns.barplot(x=day_index[::],y=day_values[::], palette = "pastel")

In [ ]:
#En çok Cumartesi günleri ufo görülmüş.
data['weekday'] = data['datetime'].dt.day_name()
order = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
weekday_data = data['weekday'].value_counts()
weekday_index = weekday_data.index 
weekday_values = weekday_data.values
plt.figure(figsize=(15,8))
plt.title('UFO Sightings by Day of the Week', fontsize=18)
plt.xlabel("Day of the week", fontsize=14)
plt.ylabel("Number of reports", fontsize=14)
sns.set_palette("pastel")
weekday_plot = sns.barplot(x=weekday_index[:60],y=weekday_values[:60], order = order)

* Cumartesi ve pazar günlerinde fazla görülme sebebinin yine dışardaki insan sayısının artışı ile ilişkili olduğunu düşünüyorum.

In [ ]:
# Hangi saatlerde raporlama artmış?
hour_data = data['hour'].value_counts() 
hour_index = hour_data.index 
hour_values =hour_data.values
plt.figure(figsize=(15,8))
plt.title('UFO Sightings by Hour', fontsize=18)
plt.xlabel("Time", fontsize=14)
plt.ylabel("Number of reports", fontsize=14)
hour_plot = sns.barplot(x=hour_index[:60],y=hour_values[:60], palette = 'Purples')

* 20-23 arası UFO raporlama sayısı ciddi şekilde artmış. Gökyüzü karanlıkken emin olamadığımız cisimleri bile daha net görürüz. Bu yüzden akşam saatlerinde UFO görünümü daha sık.

In [ ]:
data.shape

In [ ]:
data.head().T


In [ ]:
sns.set(style='darkgrid')
plt.figure(figsize=(20,10))
ax = sns.countplot(y='shape', data=data, \
                   order = data['shape'].value_counts().index).\
set_title('Gökyüzünde Görülen Cisimlerin Görülme Sıklık Grafiği')
#plt.xticks(rotation=90)

plt.ylabel('Şekil Türü', fontsize=20)
plt.xlabel('Number', fontsize=20)

* Bir ışık olduğundan bahsetmenin dışında bakarsak, triangle ve circle görenler diğerlerinden oldukça fazla. Şeklinin neye benzediğinin tanımlanmadığı durumlar da fazla.

In [ ]:

import plotly.express as px
figure = px.scatter(data,"hour","shape",color = "shape", size = "hour", hover_data = ["city"])
figure.show()

* İstisna olarak: pyramid,round,flare,hexagon,crescent ve dome şekilleri ortaya çıkıyor.
* shape round olan ve shape dome olan nadir UFO'lar aynı saatte (23) Aynı eyaletten görülmüş.
* şekli round ve dome olanların diğer verilerini karşılaştırırsak daha iyi sonuç elde edebiliriz.

In [ ]:
data.columns

In [ ]:
data[data.city=='sewickly']


In [ ]:
data[data.city=='mansfield']


In [ ]:
data.iloc[31542].comments

In [ ]:
data.iloc[9698].comments

* Bu Mansfield'da görülen UFO haberini internette araştırdığımda 200 yıl önce Mansfield'da ufo gören biri hakkında yazılmış makale ile çok yakın yorumda bulunduğunu farkettim. O yüzden Mansfield'da farklı zamanlarda görülen UFO'ların yorumlarına bakmakta fayda var.
[Masfield 200 years age ufo sighting](https://www.mansfieldnewsjournal.com/story/news/local/2014/10/27/author-investigates-year-old-ufo-sighting/18026911/) 
* Böyle bir benzerliğin olması ya bu haberin taklit edildiğini ya da gerçekten görüldüğünü düşündürüyor.

In [ ]:
fireball_man = [data[data.city=='mansfield'].comments]

In [ ]:
mansfield_comments = pd.DataFrame(fireball_man).T

In [ ]:
mansfield_comments

In [ ]:
masfield_comments_list = mansfield_comments.values.tolist()

In [ ]:
masfield_comments_list

In [ ]:
fires = []
for sub_list in masfield_comments_list:
    for string in sub_list:
        if 'fire' in string:
            fires.append(string)
print(fires)
        

In [ ]:
mans = []
for sub_list in masfield_comments_list:
    for string in sub_list:
        if 'fire' in string:
            mans.append(string)
print(mans)
        

In [ ]:
data["shape"].unique()


In [ ]:
years = data.year.unique()
years.sort()
years

In [ ]:
ax = sns.catplot(y='state',
            data=data,
            kind='count',
            height=15,
            order = data['state'].value_counts().index)

ax.fig.suptitle('Eyaletlere Göre Ufo Görme Sıklık Grafiği', x=0.5, y=1,fontsize = 25)

plt.ylabel('Eyalet', fontsize=20)
plt.xlabel('Number', fontsize=20)

* California eyaleti açık ara ile raporlamada birinci. Sonrasında yarısından da az raporlama olan Washington geliyor. 1906 yılından 2014 yılına kadar olan nüfus değişikliklerine baktığımızda:
* CA 1906 --> 1.976.000  2014 --> 38.253.768 
* WA 1906 --> 904.000  2014 --> 7.057.531
* California eyaletinin nüfusu açık ara diğerlerinden fazla. Bu nedenle UFO raporunun daha fazla olması normal. Anormal bir duruma sahip olduğunu düşünmüyorum.

In [ ]:
sns.set(style='darkgrid')
plt.figure(figsize=(20,10))
ax = sns.countplot(x='country', data=data,\
                  order = data['country'].value_counts().index, log=True).\
set_title('Ülkelere Göre UFo Raporlama Sıklık Grafiği', fontsize=25)

plt.ylabel('Number, log', fontsize=20)
plt.xlabel('Ülke', fontsize=20)

* Amerika'nın UFO Report oranı açık ara ile diğer ülkelerden fazla.

In [ ]:
sample =data.loc[(~data['latitude'].isna())].sample(10000)

Map = folium.Map(zoom_start=2)
for lat, long in zip(sample['latitude'], sample['longitude ']):
    folium.Circle((lat, long),
                   radius=0.1,
                   color='red',
                   fill_color='#3186cc',
                   ).add_to(Map)
Map

In [ ]:
sample =data.loc[ (data['country'] == 'us') & (~data['latitude'].isna())].sample(5000)

Map = folium.Map(location=(41.526653, -101.103882), zoom_start=2.7)
for lat, long in zip(sample['latitude'], sample['longitude ']):
    folium.Circle((lat, long),
                   radius=0.01,
                   color='red',
                   fill_color='#3186cc',
                   ).add_to(Map)
Map

In [ ]:
words = [i for i in data.comments.dropna()]
    
words = " ".join(words)

plt.subplots(figsize=(28,12))
wordcloud = WordCloud(
                          background_color='white',
                          width=2048,
                          height=1024
                          ).generate(words)
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('graph.png')

plt.show()

* Yapılan yorumlardan duygu analizi yapmaya çalıştım. Bunun için bert-base-multilingual-uncased-sentiment modelini kullandım.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [ ]:
result = model(tokens)


In [ ]:
result.logits


In [ ]:
tokens[0]

In [ ]:
tokenizer.decode(tokens[0])

In [ ]:
result = model(tokens)

In [ ]:
result

In [ ]:
int(torch.argmax(result.logits))+1


In [ ]:
Comments = [i for i in data["comments"]]

In [ ]:
Comments[0]

In [ ]:
df = pd.DataFrame(np.array(Comments), columns=['comments'])

In [ ]:
df

In [ ]:
df.comments.iloc[53]

In [ ]:
def sentiment_score(comments):
    tokens = tokenizer.encode(comments, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df["comments"].iloc[528])

In [ ]:
df["comments"].iloc[528]

In [ ]:
#df['sentiment'] = df['comments'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df